In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd "drive/MyDrive/Customer Segmentation in US"

/content/drive/MyDrive/Customer Segmentation in US


# 3_Modeling

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from src.model_selection import Modeling
from src.wrangle import wrangle
from sklearn.decomposition import PCA

In [ ]:
#reading data
data=pd.read_csv("data/data.csv")
data.head()

,DEBT,NETWORTH,HOUSES,NFIN,ASSET
0,12200.0,-6710.0,0.0,3900.0,5490.0
1,12600.0,-4710.0,0.0,6300.0,7890.0
2,15300.0,-8115.0,0.0,5600.0,7185.0
3,14100.0,-2510.0,0.0,10000.0,11590.0
4,15400.0,-5715.0,0.0,8100.0,9685.0


## model training and hyperprameter tuning

In [ ]:
m = Modeling()
sil_scores, inertia_vals = m.model_tuning(data, k_max=15)


In [ ]:
# plotting line curves of both lists
# Silhouette
fig1 = px.line(
    x=range(2, 15),
    y=sil_scores,
    title="Silhouette Score",
    labels={"x":"Number of Clusters","y":"Silhouette Score"}
)
fig1.show()

# Inertia
fig2 = px.line(
    x=range(2, 15),
    y=inertia_vals,
    title="Inertia",
    labels={"x":"Number of Clusters","y":"Inertia"}
)
fig2.show()

from the previous curves it seems the best n_clusters is 4 so we will make the final model with 3n_clusters

In [ ]:
final_model=m.model_pip(4)
final_model.fit(data)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('kmeans', KMeans(n_clusters=4, random_state=42))])

## Comunication

extract final model labels

In [ ]:
labels=final_model.named_steps["kmeans"].labels_
labels

array([1, 1, 1, ..., 0, 0, 0], dtype=int32)

Create a DataFrame xgb that contains the mean values of the features

In [ ]:
xgb = data.groupby(labels).mean()

print("xgb type:", type(xgb))
print("xgb shape:", xgb.shape)
xgb

xgb type: <class 'pandas.core.frame.DataFrame'>
xgb shape: (4, 5)


,DEBT,NETWORTH,HOUSES,NFIN,ASSET
0,205111.440049,2.314333e+05,261927.688504,3.512653e+05,4.365447e+05
1,28650.810927,1.176536e+04,14647.482838,2.825256e+04,4.041617e+04
2,669360.967742,5.819507e+05,788306.451613,1.118355e+06,1.251312e+06
3,266576.274510,1.432137e+06,339117.647059,1.295769e+06,1.698713e+06


create a side-by-side bar chart from xgb

In [ ]:
# Create side-by-side bar chart of `xgb`
fig =px.bar(
    xgb,
    barmode="group",
    title="Mean Values of Features by Cluster"
)
fig.update_layout(
    xaxis_title="Cluster",
    yaxis_title="Mean Value",
    legend_title="Feature"
)
fig.show()

Create a PCA transformer, use it to reduce the dimensionality of the data

In [ ]:
pca = PCA(n_components=2 , random_state=42)

# Transform `data`
X_t =pca.fit_transform(data)

# Put `X_t` into DataFrame
X_pca = pd.DataFrame(X_t ,columns=["Pca1" ,"Pca2"])

print("X_pca type:", type(X_pca))
print("X_pca shape:", X_pca.shape)
X_pca.head()

X_pca type: <class 'pandas.core.frame.DataFrame'>
X_pca shape: (4418, 2)


,Pca1,Pca2
0,-221525.424530,22052.273003
1,-217775.100722,22851.358068
2,-219519.642175,19023.646333
3,-212195.720367,22957.107039
4,-215540.507551,20259.749306


In [ ]:
# Create scatter plot of `PC2` vs `PC1`
fig = px.scatter(
    data_frame=X_pca,
    x="Pca1",
    y="Pca2",
    color=labels.astype(str)
)
fig.update_layout(
    title="PCA Scatter Plot",
    xaxis_title="PC1",
    yaxis_title="PC2",
    legend_title="Cluster"
)
fig.show()